In [1]:
import sys
sys.path.append('../src')

from utils.data_utils import read_training_data, read_model_zoo
from utils.pipeline_utils import run_cv
from models.tfidf_model import TfidfModel

In [2]:
df = read_training_data()
df.head()

,id,text,is_offensive,target,text_len,target_label,public_fold,private_fold,pred
0,81c11060-a240-4d54-841b-9e2916039e85,çürük dişli,1,INSULT,11,4,1,0,OTHER
1,be80ebbf-b322-4c3b-afa1-94932ea80731,bu adamın islama ve müslümanlara verdiği zarar...,1,RACIST,67,3,3,0,OTHER
2,f99e2513-83ed-4076-ac72-b9e2cff3f049,erkekler zora gelmez,1,SEXIST,20,2,2,0,OTHER
3,83ed2b2e-b815-4f36-9fc4-80a9050cf2d0,utanmazın götüne kazık sokmuşlar bu tıkırtı ne...,1,PROFANITY,65,1,4,0,OTHER
4,d93e05f7-bfdd-4cdb-99d8-3048761b30ff,otomasyon< sistemlerine= doğrudan bağlanabilir,0,OTHER,46,0,4,0,OTHER


In [3]:
result_dict = run_cv(model_obj=TfidfModel(),
                     input_df=df,
                     fold_col="public_fold",
                     x_col="text",
                     y_col="target",
                     experiment_name="TFIDF + LGB",
                     add_to_zoo=True
                    )

Training.. Fold:   0%|          | 0/5 [00:00<?, ?it/s]

              precision    recall  f1-score   support

      INSULT       0.59      0.56      0.57      2393
       OTHER       0.55      0.82      0.66      3528
   PROFANITY       0.88      0.59      0.71      2376
      RACIST       0.75      0.51      0.61      2033
      SEXIST       0.84      0.77      0.81      2081

    accuracy                           0.67     12411
   macro avg       0.72      0.65      0.67     12411
weighted avg       0.70      0.67      0.67     12411

The result is logged to the model zoo!
